In [1]:
import bw2data as bd
import bw2calc as bc
import bw_processing as bwp

In [2]:
bd.projects.set_current('GSA for archetypes')

In [3]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.8 cutoff
	swiss consumption 1.0

# Local sensitivity anaysis

We want to be able to change one exchange by a constant factor and test the effect on LCA scores.

The most efficient way to do this is with an `Interface`, so we don't have to generate many near-identical columns with only one element changing.

Our interface will be based on the `ecoinvent` data package, so let's start with that.

In [4]:
ei = bd.Database('ecoinvent 3.8 cutoff').datapackage()
ei

datapackages can have multiple resources; we only need the technosphere (for now)

In [5]:
ei = ei.filter_by_attribute('matrix', 'technosphere_matrix')

In [6]:
[x['name'] for x in ei.resources]

['ecoinvent_3.8_cutoff_technosphere_matrix.indices',
 'ecoinvent_3.8_cutoff_technosphere_matrix.data',
 'ecoinvent_3.8_cutoff_technosphere_matrix.distributions',
 'ecoinvent_3.8_cutoff_technosphere_matrix.flip']

Construct an `Interface` that iteratively changes every exchange with uncertainty by `factor`.

In [7]:
class LocalSAInterface:
    def __init__(self, indices, data, distributions, factor=10):
        self.indices = indices
        self.data = data
        self.distributions = distributions
        self.factor = factor

        assert self.indices.shape[0] == self.data.shape[0] == self.distributions.shape[0]
        
        self.size = len(self.indices)
        self.index = None  # To indicate we haven't consumed first value yet
        
    def __next__(self):
        if self.index is None:
            self.index = 0
        else:
            self.index += 1
            
        while self.distributions[self.index]['uncertainty_type'] < 2: # 0 and 1 are no and unknown uncertainty                             
            self.index += 1
            if self.index >= self.size:
                raise StopIteration

        data = self.data.copy()
        data[self.index] *= self.factor
        return data
    
    @property
    def coordinates(self):
        return self.indices[self.index]

In [8]:
tech_interface = LocalSAInterface(
    ei.get_resource('ecoinvent_3.8_cutoff_technosphere_matrix.indices')[0],
    ei.get_resource('ecoinvent_3.8_cutoff_technosphere_matrix.data')[0],
    ei.get_resource('ecoinvent_3.8_cutoff_technosphere_matrix.distributions')[0],
    10
)

The interface needs to be in a datapackage; we create one in memory.

In [9]:
dp = bwp.create_datapackage()
dp.add_dynamic_vector(
    matrix = 'technosphere_matrix',
    interface = tech_interface,
    indices_array = ei.get_resource('ecoinvent_3.8_cutoff_technosphere_matrix.indices')[0],
    flip_array = ei.get_resource('ecoinvent_3.8_cutoff_technosphere_matrix.flip')[0],  # Only needed for technosphere
)

How do we use this datapackage? We can just append it to the list of datapackages. First, the setup for the static LCA:

In [10]:
fu = {bd.Database('ecoinvent 3.8 cutoff').random(): 1}

In [11]:
fu_mapped, packages, _ = bd.prepare_lca_inputs(demand=fu, remapping=False)  # Could also add LCIA method

In [12]:
lca = bc.LCA(demand=fu_mapped, data_objs=packages)

In [13]:
lca.lci()
lca.inventory.sum()

13692.361296438652

And now with our local SA interface:

In [14]:
lca2 = bc.LCA(demand=fu_mapped, data_objs=packages + [dp])

In [ ]:
lca2.lci()

In [ ]:
for x in range(10):
    next(lca2) # Will also do lci and lcia calculation as needed
    i, j = tech_interface.coordinates
    row, col = lca2.dicts.product[i], lca2.dicts.activity[j]
    print(
        lca.inventory.sum(), 
        tech_interface.index, 
        tech_interface.coordinates, row, col,
        lca2.technosphere_matrix[row, col], 
        lca.technosphere_matrix[row, col] 
    )
    # product = bd.get_activity(id=i)
    # activity = bd.get_activity(id=j)
    # print(product, activity)

Actually calculating the local SA indices with a LCIA method is left as an exercise for the reader :)